In [ ]:
using JLD2
using Statistics
using Revise
using Plots
using Laplacians
using Printf

In [ ]:
function sd(d,x)
    #scale = exp10(ceil(log10(abs(x)))-d)
    #return round(x/scale)*scale
    return round(x, sigdigits=d)
end        
ff(s) = (@sprintf "%.4g" sd(3,s))
ffK(s) = (@sprintf "%.5g" sd(3,s)/1e3)
ffM(s) = (@sprintf "%.3g" sd(3,s)/1e6)
ffmicro(s) = (@sprintf "%.4g" sd(3,s)*1e6)
ffe6(s) = (@sprintf "%.3g" sd(3,s)*1e6)
ffe8(s) = (@sprintf "%.4g" sd(3,s)*1e8)

In [ ]:
@load "variances.jld2"
dic1 = copy(dic);

In [ ]:
wted_chimera_indices = findall(entry -> contains(entry, "wted_chimera(10000000,1)"), dic1["testName"])
sachedeva_indices = findall(entry -> contains(entry, "star_join(complete_graph(700), 350)"), dic1["testName"])
wted_bndry_chimera_indices = findall(entry -> contains(entry, "wted_bndry_chimera(10000000,1)"), dic1["testName"])
checkered_grid_indices = findall(entry -> contains(entry, "checkered_grid_sddm(2.0e8, 32, 32, 32, 1.0e7)"), dic1["testName"])
wgrid_indices = findall(entry -> contains(entry, "wgrid_sddm(2.0e8, 0.001)"), dic1["testName"]);

In [ ]:
problem_idx = [wted_chimera_indices, wted_bndry_chimera_indices, sachedeva_indices, checkered_grid_indices, wgrid_indices]
instanceNames = ["Weighted chimera", "Weighted SDDM chimera", "Sachdeva Star", "High contrast coefficient Poisson grid", "Anisotropic coef. Poisson grid, variable weight"];


In [ ]:
function extractSlice(dic, slice)
    dicSlice = Dict()
    numepxr = length(dic["ne"])
    for (key, value) in dic
        if length(value) == numepxr
            dicSlice[key] = value[slice]
        else
            dicSlice[key] = value
        end
    end
    return dicSlice
end

In [ ]:
dics = [extractSlice(dic1, idx) for idx in problem_idx];

In [ ]:
instanceNnzs = [maximum(dic1["ne"][indices]) for indices in problem_idx]


In [ ]:
function labelTvn(tvn, err)
    if err <= 1e-8
        return ffe6(tvn)
    elseif err < 1e-4
        return string(ffe6(tvn), "\\tnote{*}")
    elseif err < 1
        return string(ffe6(tvn), "\\tnote{**}")
    else
        return "Inf"
    end
end

function printMedian(d, solver, category)
    if haskey(d, string(solver, category)) && length(d[string(solver, category)]) == length(d["ne"])
        tvn = median(d[string(solver, category)] ./ d["ne"] )
        err = median(d[string(solver, "_err")])
        return labelTvn(tvn, err)
    else
        return "N/A"
    end
end

function print75(d, solver, category)
    if haskey(d, string(solver, category)) && length(d[string(solver, category)]) == length(d["ne"])
        tvn = quantile(d[string(solver, category)] ./ d["ne"], 0.75)
        err = quantile(d[string(solver, "_err")], 0.75)
        return labelTvn(tvn, err)
    else
        return "N/A"
    end
end

function printMax(d, solver, category)
    if haskey(d, string(solver, category)) && length(d[string(solver, category)]) == length(d["ne"])
        tvn = maximum(d[string(solver, category)] ./ d["ne"])
        err = maximum(d[string(solver, "_err")])
        return labelTvn(tvn, err)
    else
        return "N/A"
    end
end

In [ ]:
function printSolverCell(d, solver, category) 
    return string(printMedian(d, solver, category), "&", print75(d, solver, category), "&", printMax(d, solver, category))
end

function printOneLine(d, solvers)
    allSolveCells = [printSolverCell(d, solver, "_solve") for solver in solvers]
    allTotCells = [printSolverCell(d, solver, "_tot") for solver in solvers]
    return string(join(allSolveCells, "&"), "&", join(allTotCells, "&"), "\\\\")
end

function printAllLines(instanceNames, instanceNnzs, dics, solvers)
    for i in 1:length(dics)
        println(instanceNames[i], "&", ffM(instanceNnzs[i]) , "&", printOneLine(dics[i], solvers))
    end
end

In [ ]:
solvers = ["ac", "ac-s2m2"]
printAllLines(instanceNames, instanceNnzs, dics, solvers)

In [ ]:
solvers = ["ac", "ac-random"]
printAllLines(instanceNames, instanceNnzs, dics, solvers)